# 🎵 Spotify Playlist & Album Downloader by Makuguren

**Complete Spotify Downloader with High-Quality Audio & Metadata**

This downloader provides robust playlist and album downloading with comprehensive error handling.

---

## 🎯 Features:
- **Playlist & Album support** - Download both playlists and full albums
- **High-quality audio** downloads (up to 320kbps)
- **Complete metadata** including album art and lyrics
- **Flexible fallback options** for maximum success rate
- **Smart error handling** and recovery
- **Multiple export options** (Google Drive, ZIP downloads)

---

**💡 Download your favorite Spotify playlists and albums with ease!**


In [ ]:
# Install required packages
!pip install spotdl yt-dlp --quiet
!spotdl --download-ffmpeg
print("✅ Installation complete!")

In [ ]:
import os
import json
import re
import subprocess
from pathlib import Path
from typing import List, Dict, Optional, Any
import tempfile
from IPython.display import clear_output
import time

def get_value(obj: Any, key: str, default: Any = None) -> Any:
    """Safely get values from objects with fallback support"""
    try:
        if isinstance(obj, dict):
            return obj.get(key, default)
        elif hasattr(obj, key):
            return getattr(obj, key, default)
        else:
            return default
    except:
        return default

def get_nested_value(obj: Any, *keys: str, default: Any = None) -> Any:
    """Get nested values with safe navigation"""
    try:
        current = obj
        for key in keys:
            if isinstance(current, dict):
                current = current.get(key, {})
            elif hasattr(current, key):
                current = getattr(current, key, {})
            else:
                return default
        return current if current != {} else default
    except:
        return default

def format_list_as_string(obj: Any, separator: str = ", ") -> str:
    """Convert objects to readable string format"""
    try:
        if isinstance(obj, list):
            return separator.join(str(item) for item in obj if item)
        elif isinstance(obj, str):
            return obj
        else:
            return str(obj) if obj else "Unknown"
    except:
        return "Unknown"

class SpotifyPlaylistDownloader:
    def __init__(self, output_dir: str = "Downloads"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        print(f"🎵 Spotify Playlist & Album Downloader initialized!")
    
    def clean_filename(self, filename: str) -> str:
        """Clean filename for filesystem compatibility"""
        try:
            filename = re.sub(r'[<>:"/\\|?*]', '_', str(filename))
            filename = re.sub(r'[^\w\s\-_\.]', '_', filename)
            return filename.strip()[:200]
        except:
            return "unknown_file"
    
    def is_valid_spotify_url(self, url: str) -> bool:
        """Validate Spotify playlist or album URL"""
        try:
            url_lower = str(url).lower()
            return ('spotify.com/playlist/' in url_lower or 
                    'spotify.com/album/' in url_lower)
        except:
            return False
    
    def get_url_type(self, url: str) -> str:
        """Determine if URL is playlist or album"""
        try:
            url_lower = str(url).lower()
            if 'spotify.com/playlist/' in url_lower:
                return "playlist"
            elif 'spotify.com/album/' in url_lower:
                return "album"
            else:
                return "unknown"
        except:
            return "unknown"
    
    def fetch_playlist_data(self, spotify_url: str) -> Optional[Dict]:
        """Fetch playlist or album metadata using spotdl"""
        try:
            url_type = self.get_url_type(spotify_url)
            print(f"🔍 Fetching {url_type} metadata...")
            
            temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.spotdl', delete=False)
            temp_file.close()
            
            cmd = ['spotdl', 'save', str(spotify_url), '--save-file', temp_file.name]
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            
            with open(temp_file.name, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            os.unlink(temp_file.name)
            
            # Handle both dictionary and list formats from spotdl
            if isinstance(data, dict):
                print("✅ Metadata loaded successfully (dictionary format)")
                return data
            elif isinstance(data, list):
                print("🔧 Converting list format to expected structure")
                collection_name = self.extract_collection_name(spotify_url)
                converted_data = {
                    'name': collection_name,
                    'songs': data,
                    'url': spotify_url,
                    'type': url_type
                }
                print(f"✅ Processed {len(data)} songs successfully")
                return converted_data
            else:
                print(f"⚠️ Warning: Unexpected data format: {type(data)}")
                return None
            
        except Exception as e:
            print(f"❌ Error fetching metadata: {e}")
            return None
    
    def extract_collection_name(self, url: str) -> str:
        """Extract or generate collection name from URL (playlist or album)"""
        try:
            url_type = self.get_url_type(url)
            if 'playlist/' in url:
                playlist_id = url.split('playlist/')[-1].split('?')[0]
                return f"Spotify_Playlist_{playlist_id[:8]}"
            elif 'album/' in url:
                album_id = url.split('album/')[-1].split('?')[0]
                return f"Spotify_Album_{album_id[:8]}"
            else:
                return f"Spotify_{url_type.title()}"
        except:
            return "Downloaded_Collection"
    
    def extract_track_details(self, track: Any) -> tuple:
        """Extract track information from various data structures"""
        try:
            title = "Unknown"
            artists = []
            spotify_url = ""
            
            # Handle different track data structures
            if isinstance(track, dict):
                title = get_value(track, 'name', 'Unknown')
                artists = get_value(track, 'artists', [])
                spotify_url = get_value(track, 'url', '')
                
            elif isinstance(track, list) and len(track) > 0:
                first_item = track[0] if track else {}
                if isinstance(first_item, dict):
                    title = get_value(first_item, 'name', 'Unknown')
                    artists = get_value(first_item, 'artists', [])
                    spotify_url = get_value(first_item, 'url', '')
                else:
                    title = format_list_as_string(track)
            
            # Ensure valid title
            if not title or title == "Unknown":
                title = f"Track_{id(track)}"
            
            artist_str = format_list_as_string(artists)
            return title, artist_str, spotify_url
            
        except Exception as e:
            print(f"⚠️ Error extracting track details: {e}")
            return "Unknown", "Unknown", ""
    
    def download_with_ytdlp_fallback(self, video_url: str, output_path: str, track_info: tuple) -> bool:
        """Fallback download with yt-dlp for high quality audio"""
        try:
            title, artist_str, _ = track_info
            
            cmd = [
                'yt-dlp',
                '--extract-audio',
                '--audio-format', 'mp3',
                '--audio-quality', '320K',
                '--embed-metadata',
                '--add-metadata',
                '--no-warnings',
                '--write-thumbnail',
                '--embed-thumbnail',
                '-o', str(output_path),
                str(video_url)
            ]
            
            print(f"⬇️ Fallback download: {artist_str} - {title}")
            subprocess.run(cmd, capture_output=True, text=True, check=True)
            return True
            
        except Exception:
            return False
    
    def download_with_spotdl_primary(self, track_url: str, output_dir: str, track_info: tuple) -> bool:
        """Primary download method using spotdl"""
        try:
            title, artist_str, _ = track_info
            
            cmd = [
                'spotdl', 'download', str(track_url),
                '--output', str(output_dir),
                '--format', 'mp3',
                '--bitrate', '320k',
                '--embed-metadata',
                '--generate-lrc',
                '--save-cover' # This should include album art
            ]
            
            print(f"⬇️ Downloading: {artist_str} - {title}")
            subprocess.run(cmd, capture_output=True, text=True, check=True)
            return True
            
        except Exception:
            # Try with fallback options
            try:
                cmd_fallback = [
                    'spotdl', 'download', str(track_url),
                    '--output', str(output_dir),
                    '--format', 'mp3',
                    '--bitrate', '320k'
                ]
                subprocess.run(cmd_fallback, capture_output=True, text=True, check=True)
                return True
            except Exception:
                return False
    
    def search_youtube_track(self, artist: str, title: str) -> Optional[str]:
        """Search YouTube for track using yt-dlp"""
        try:
            search_query = f"ytsearch3:{artist} - {title}"
            cmd = ['yt-dlp', '--quiet', '--dump-json', '--playlist-end', '3', search_query]
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            
            for line in result.stdout.strip().split('\n'):
                if line:
                    try:
                        video = json.loads(line)
                        return get_value(video, 'webpage_url')
                    except:
                        continue
            return None
        except:
            return None
    
    def download_collection(self, spotify_url: str) -> bool:
        """Download complete playlist or album with error handling"""
        try:
            url_type = self.get_url_type(spotify_url)
            print(f"🎵 Processing {url_type}: {spotify_url}")
            
            # Get metadata
            metadata = self.fetch_playlist_data(spotify_url)
            if not metadata:
                print(f"❌ Failed to fetch {url_type} metadata")
                return False
            
            # Extract collection information
            collection_name = get_value(metadata, 'name', f'Unknown {url_type.title()}')
            tracks = get_value(metadata, 'songs', [])
            
            if not tracks:
                print(f"❌ No tracks found in {url_type}")
                return False
            
            print(f"📂 {url_type.title()}: {collection_name}")
            print(f"🎵 Total tracks: {len(tracks)}")
            
            # Create output folder
            folder = self.output_dir / self.clean_filename(collection_name)
            folder.mkdir(exist_ok=True)
            
            # Save metadata
            try:
                metadata_filename = f"{url_type}_info.json"
                with open(folder / metadata_filename, 'w', encoding='utf-8') as f:
                    json.dump(metadata, f, indent=2, ensure_ascii=False)
            except:
                print("⚠️ Could not save metadata file")
            
            success_count = 0
            failed_count = 0
            
            # Process each track
            for i, track in enumerate(tracks, 1):
                try:
                    # Extract track information
                    title, artist_str, spotify_url = self.extract_track_details(track)
                    
                    current_track = f"{artist_str} - {title}"
                    print(f"[{i}/{len(tracks)}] Processing: {current_track}")
                    
                    filename = self.clean_filename(f"{artist_str} - {title}")
                    output_path = folder / f"{filename}.%(ext)s"
                    
                    # Check if file already exists
                    existing_files = list(folder.glob(f"{filename}.*"))
                    if existing_files:
                        success_count += 1
                        print("⏭️ Already exists, skipping...")
                        continue
                    
                    # Try download methods
                    success = False
                    track_info = (title, artist_str, spotify_url)
                    
                    # Primary: spotdl download
                    if spotify_url:
                        success = self.download_with_spotdl_primary(spotify_url, str(folder), track_info)
                    
                    # Fallback: yt-dlp search and download
                    if not success:
                        youtube_url = self.search_youtube_track(artist_str, title)
                        if youtube_url:
                            success = self.download_with_ytdlp_fallback(youtube_url, str(output_path), track_info)
                    
                    if success:
                        success_count += 1
                        print("✅ Downloaded successfully")
                    else:
                        failed_count += 1
                        print("❌ Download failed")
                
                except Exception as e:
                    print(f"⚠️ Error processing track {i}: {e}")
                    failed_count += 1
                    continue
            
            print(f"\n🎵 Download Complete!")
            print(f"📊 Summary: {success_count} successful, {failed_count} failed")
            print(f"📁 Files saved to: {folder}")
            
            return success_count > 0
            
        except Exception as e:
            print(f"❌ Critical error in {url_type} download: {e}")
            return False
    
    # Keep the old method name for backwards compatibility
    def download_playlist(self, playlist_url: str) -> bool:
        """Download playlist (wrapper for download_collection)"""
        return self.download_collection(playlist_url)

# Initialize the downloader
print("🎵 Spotify Playlist & Album Downloader ready!")
downloader = SpotifyPlaylistDownloader()


In [ ]:
# Interactive download interface
def interactive_playlist_download():
    """Interactive interface for downloading playlists and albums"""
    
    print("🎵 SPOTIFY PLAYLIST & ALBUM DOWNLOADER")
    print("=" * 50)
    print("Download your favorite Spotify playlists and albums with high-quality audio!")
    print()
    print("📝 Supported URLs:")
    print("   • Spotify Playlists: https://open.spotify.com/playlist/...")
    print("   • Spotify Albums: https://open.spotify.com/album/...")
    print()
    
    while True:
        print("🎯 Enter Spotify Playlist or Album URL (or 'done' to finish):")
        user_input = input("URL: ").strip()
        
        if user_input.lower() in ['done', 'exit', 'quit', '']:
            break
        
        if not downloader.is_valid_spotify_url(user_input):
            print("❌ Invalid Spotify URL! Please enter a valid playlist or album URL.")
            continue
        
        url_type = downloader.get_url_type(user_input)
        print(f"🎵 Starting {url_type} download...")
        
        try:
            success = downloader.download_collection(user_input)
            
            if success:
                print(f"\n✅ {url_type.title()} download completed successfully!")
            else:
                print(f"\n❌ {url_type.title()} download failed!")
                
        except Exception as e:
            print(f"\n❌ Error occurred: {e}")
        
        print("\n" + "="*50)
    
    print("🎵 Download session complete!")

# Run the interactive downloader
interactive_playlist_download()


## 📂 Browse Your Downloads

Check what playlists, albums and songs you've downloaded:

In [ ]:
def browse_downloaded_playlists():
    """Browse and display downloaded files"""
    downloads_dir = Path("Downloads")
    
    if not downloads_dir.exists():
        print("❌ No downloads folder found")
        print("📝 Run the download cell first to create some downloads!")
        return
    
    collections = [p for p in downloads_dir.iterdir() if p.is_dir()]
    
    if not collections:
        print("📭 No playlists or albums downloaded yet")
        print("📝 Run the download cell to start downloading playlists and albums!")
        return
    
    print("📂 YOUR DOWNLOADED PLAYLISTS & ALBUMS")
    print("=" * 50)
    
    total_songs = 0
    total_size = 0
    
    for i, collection_folder in enumerate(collections, 1):
        try:
            # Determine collection type from folder name or metadata
            collection_type = "Collection"
            if "playlist" in collection_folder.name.lower():
                collection_type = "🎵 Playlist"
            elif "album" in collection_folder.name.lower():
                collection_type = "💿 Album"
            else:
                # Check for metadata files to determine type
                if (collection_folder / "playlist_info.json").exists():
                    collection_type = "🎵 Playlist"
                elif (collection_folder / "album_info.json").exists():
                    collection_type = "💿 Album"
                else:
                    collection_type = "📁 Collection"
            
            # Count songs (exclude metadata file)
            songs = [f for f in collection_folder.iterdir() 
                    if f.suffix.lower() in ['.mp3', '.flac', '.m4a'] and f.is_file()]
            
            # Count additional files (lyrics, covers)
            lrc_files = [f for f in collection_folder.iterdir() if f.suffix.lower() == '.lrc']
            cover_files = [f for f in collection_folder.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
            
            # Calculate folder size
            folder_size = 0
            try:
                folder_size = sum(f.stat().st_size for f in collection_folder.rglob('*') if f.is_file())
            except:
                pass
            folder_size_mb = folder_size / (1024 * 1024) if folder_size > 0 else 0
            
            print(f"{collection_type} {i}. {collection_folder.name}")
            print(f"   📊 Songs: {len(songs)}")
            print(f"   📜 Lyrics: {len(lrc_files)} files")
            print(f"   🖼️ Album art: {len(cover_files)} covers")
            print(f"   💾 Size: {folder_size_mb:.1f} MB")
            
            # Show sample tracks
            if songs:
                print("   🎶 Sample tracks:")
                for song in songs[:3]:
                    song_name = song.name
                    if len(song_name) > 60:
                        song_name = song_name[:57] + "..."
                    print(f"      • {song_name}")
                if len(songs) > 3:
                    print(f"      ... and {len(songs) - 3} more")
            else:
                print("   ⚠️ No audio files found")
            
            total_songs += len(songs)
            total_size += folder_size_mb
            print()
            
        except Exception as e:
            print(f"⚠️ Error processing {collection_folder.name}: {e}")
            continue
    
    print("📊 DOWNLOAD SUMMARY:")
    print(f"   📁 Total collections: {len(collections)}")
    print(f"   🎶 Total songs: {total_songs}")
    print(f"   💾 Total size: {total_size:.1f} MB")
    print(f"   📍 Location: {downloads_dir.absolute()}")
    print("=" * 50)

# Browse your downloads
browse_downloaded_playlists()


## 💾 Save to Google Drive (Optional)

Mount Google Drive and copy your downloads:


In [ ]:
def backup_to_google_drive():
    """Backup downloads to Google Drive with error handling"""
    try:
        from google.colab import drive
        import shutil
        
        print("💾 GOOGLE DRIVE BACKUP")
        print("=" * 50)
        print("🔑 You'll be asked to authenticate with Google...")
        
        # Mount Google Drive
        try:
            drive.mount('/content/drive')
            print("✅ Google Drive mounted successfully!")
        except Exception as e:
            print(f"❌ Failed to mount Google Drive: {e}")
            return
        
        # Check if downloads exist
        source_dir = "/content/Downloads"
        if not os.path.exists(source_dir):
            print("❌ No downloads folder found!")
            print("📝 Download some playlists first, then run this cell.")
            return
        
        # Create destination directory
        drive_dir = "/content/drive/MyDrive/Spotify_Downloads"
        try:
            os.makedirs(drive_dir, exist_ok=True)
            print(f"📁 Created/verified directory: {drive_dir}")
        except Exception as e:
            print(f"❌ Could not create directory: {e}")
            return
        
        print(f"📁 Copying downloads to Google Drive...")
        print(f"📍 Source: {source_dir}")
        print(f"📍 Destination: {drive_dir}")
        
        copied_playlists = 0
        copied_files = 0
        
        # Copy each playlist folder
        try:
            for item in os.listdir(source_dir):
                source_path = os.path.join(source_dir, item)
                drive_path = os.path.join(drive_dir, item)
                
                if os.path.isdir(source_path):
                    try:
                        print(f"📂 Copying playlist: {item}")
                        
                        # Remove existing folder if it exists
                        if os.path.exists(drive_path):
                            shutil.rmtree(drive_path)
                        
                        # Copy the folder
                        shutil.copytree(source_path, drive_path)
                        copied_playlists += 1
                        
                        # Count files in the copied folder
                        try:
                            folder_files = len([f for f in os.listdir(drive_path) if os.path.isfile(os.path.join(drive_path, f))])
                            copied_files += folder_files
                            print(f"   ✅ Copied {folder_files} files successfully!")
                        except:
                            print(f"   ✅ Copied successfully!")
                            
                    except Exception as e:
                        print(f"   ❌ Error copying {item}: {e}")
                        continue
        
        except Exception as e:
            print(f"❌ Error accessing source directory: {e}")
            return
        
        print("\\n🎉 BACKUP COMPLETE!")
        print("=" * 50)
        print(f"✅ Copied {copied_playlists} playlist(s) to Google Drive")
        print(f"📁 Total files: {copied_files}")
        print(f"📍 Location: MyDrive/Spotify_Downloads/")
        print("💡 You can now access your music from any device!")
        print("=" * 50)
        
    except ImportError:
        print("❌ This feature is only available in Google Colab")
        print("💡 You can manually copy files or use the ZIP download option")
    except Exception as e:
        print(f"❌ Unexpected error in Google Drive backup: {e}")

# Ask user if they want to save to Google Drive
print("💾 Do you want to backup your downloads to Google Drive?")
print("This will copy all your downloaded playlists and albums with album art and lyrics!")
choice = input("Type 'yes' to continue or 'no' to skip: ").strip().lower()

if choice in ['yes', 'y', '1']:
    backup_to_google_drive()
else:
    print("⏭️ Skipped Google Drive backup")
    print("💡 You can always run this cell later to backup your downloads")


## 📦 Download as ZIP (Computer Download)

Create ZIP files and download directly to your computer:


In [ ]:
def create_zip_downloads():
    """Create ZIP files for downloading to computer"""
    try:
        import zipfile
        from google.colab import files
        
        print("📦 ZIP DOWNLOADER")
        print("=" * 50)
        
        downloads_dir = "Downloads"
        if not os.path.exists(downloads_dir):
            print("❌ No downloads folder found!")
            print("📝 Download some playlists first before creating ZIP files.")
            return
        
        # Get list of available playlists
        playlists = [d for d in os.listdir(downloads_dir) 
                    if os.path.isdir(os.path.join(downloads_dir, d))]
        
        if not playlists:
            print("❌ No playlists found to ZIP!")
            return
        
        print("🎵 Available playlists:")
        for i, playlist in enumerate(playlists, 1):
            # Count files
            try:
                playlist_path = os.path.join(downloads_dir, playlist)
                file_count = len([f for f in os.listdir(playlist_path) 
                                if os.path.isfile(os.path.join(playlist_path, f))])
                print(f"   {i}. {playlist} ({file_count} files)")
            except:
                print(f"   {i}. {playlist}")
        
        print("\\n📦 Choose what to download:")
        print("   • Type 'all' to download everything as one ZIP")
        print("   • Type numbers (e.g., '1,3,5') to download specific playlists")
        print("   • Type a number for individual playlist ZIP")
        
        choice = input("\\nYour choice: ").strip().lower()
        
        if choice == 'all':
            # Create one ZIP with all playlists
            zip_name = "Spotify_All_Playlists.zip"
            create_single_zip(downloads_dir, zip_name, "all playlists")
            
        elif ',' in choice:
            # Multiple specific playlists
            try:
                indices = [int(x.strip()) - 1 for x in choice.split(',')]
                selected_playlists = [playlists[i] for i in indices if 0 <= i < len(playlists)]
                
                if selected_playlists:
                    zip_name = f"Spotify_Selected_Playlists.zip"
                    create_selective_zip(downloads_dir, selected_playlists, zip_name)
                else:
                    print("❌ No valid playlists selected!")
            except:
                print("❌ Invalid selection format!")
                
        else:
            # Single playlist
            try:
                index = int(choice) - 1
                if 0 <= index < len(playlists):
                    selected_playlist = playlists[index]
                    safe_name = selected_playlist.replace(' ', '_').replace('/', '_')
                    zip_name = f"Spotify_{safe_name}.zip"
                    
                    playlist_path = os.path.join(downloads_dir, selected_playlist)
                    create_single_zip(playlist_path, zip_name, f"playlist '{selected_playlist}'")
                else:
                    print("❌ Invalid playlist number!")
            except:
                print("❌ Please enter a valid number!")
                
    except ImportError:
        print("❌ This feature requires Google Colab environment")
    except Exception as e:
        print(f"❌ Unexpected error in ZIP creation: {e}")

def create_single_zip(source_path, zip_name, description):
    """Create ZIP file for a single source"""
    try:
        import zipfile
        
        print(f"\\n📦 Creating ZIP for {description}...")
        
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            if os.path.isfile(source_path):
                # Single file
                zipf.write(source_path, os.path.basename(source_path))
                file_count = 1
            else:
                # Directory
                file_count = 0
                for root, dirs, file_list in os.walk(source_path):
                    for file_name in file_list:
                        try:
                            file_path = os.path.join(root, file_name)
                            archive_name = os.path.relpath(file_path, source_path)
                            zipf.write(file_path, archive_name)
                            file_count += 1
                        except Exception as e:
                            print(f"⚠️ Warning: Could not add {file_name}: {e}")
                            continue
        
        # Get ZIP size
        try:
            zip_size = os.path.getsize(zip_name) / (1024 * 1024)  # MB
            print(f"✅ ZIP created successfully!")
            print(f"   📁 Files: {file_count}")
            print(f"   💾 Size: {zip_size:.1f} MB")
            print(f"   🏷️ Name: {zip_name}")
        except:
            print(f"✅ ZIP created successfully: {zip_name}")
        
        print(f"✅ ZIP file ready: {zip_name}")
        print(f"💡 You can find the ZIP file in the file browser on the left")
        print(f"📥 Right-click the file and select 'Download' to save to your computer")
        
    except Exception as e:
        print(f"❌ Error creating ZIP: {e}")

def create_selective_zip(downloads_dir, selected_playlists, zip_name):
    """Create ZIP for multiple selected playlists"""
    try:
        import zipfile
        
        print(f"\\n📦 Creating ZIP for {len(selected_playlists)} selected playlists...")
        
        file_count = 0
        
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for playlist in selected_playlists:
                playlist_path = os.path.join(downloads_dir, playlist)
                
                if os.path.exists(playlist_path):
                    print(f"   📂 Adding: {playlist}")
                    
                    for root, dirs, file_list in os.walk(playlist_path):
                        for file_name in file_list:
                            try:
                                file_path = os.path.join(root, file_name)
                                # Create archive path: playlist_name/file
                                archive_name = os.path.join(playlist, os.path.relpath(file_path, playlist_path))
                                zipf.write(file_path, archive_name)
                                file_count += 1
                            except Exception as e:
                                print(f"⚠️ Warning: Could not add {file_name}: {e}")
                                continue
                else:
                    print(f"⚠️ Warning: {playlist} not found")
        
        # Get ZIP size
        try:
            zip_size = os.path.getsize(zip_name) / (1024 * 1024)  # MB
            print(f"✅ ZIP created successfully!")
            print(f"   📁 Files: {file_count}")
            print(f"   💾 Size: {zip_size:.1f} MB")
            print(f"   🏷️ Name: {zip_name}")
        except:
            print(f"✅ ZIP created successfully: {zip_name}")
        
        print(f"✅ ZIP file ready: {zip_name}")
        print(f"💡 You can find the ZIP file in the file browser on the left")
        print(f"📥 Right-click the file and select 'Download' to save to your computer")
        
    except Exception as e:
        print(f"❌ Error creating selective ZIP: {e}")

# Ask user if they want to create ZIP downloads
print("📦 Do you want to create ZIP files for download to your computer?")
print("This will package your playlists and albums with all music, album art, and lyrics!")
choice = input("Type 'yes' to continue or 'no' to skip: ").strip().lower()

if choice in ['yes', 'y', '1']:
    create_zip_downloads()
else:
    print("⏭️ Skipped ZIP creation")
    print("💡 You can always run this cell later to create ZIP downloads")
